In [6]:
import pandas as pd

In [7]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [8]:
import warnings

warnings.simplefilter('ignore')

In [9]:
import time
import math
import re
from textblob import TextBlob
import pandas as pd

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string


import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim

In [10]:
data=pd.read_pickle('webhose_cat.pkl')

In [14]:
# Filter non-English text
text_eng = data[data['language']=='english'].reset_index(drop=True)

In [18]:
# Remove special characters to avoid problems with analysis
text_eng['text_clean'] = text_eng['text'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))

In [1]:
pd.set_option('display.max_colwidth', 100)
text_eng[['text', 'text_clean']].head(5)

NameError: name 'pd' is not defined

## Topic Modeling

### LDA (latent dirichlet allocation)

In [27]:
doc_complete=text_eng.text_clean.tolist()

In [28]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]     

In [31]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [38]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

### Three-topic Model

In [39]:
warnings.simplefilter('ignore')

# Running and Trainign LDA model on the document term matrix.
%time ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50) #3 topics
print(*ldamodel.print_topics(num_topics=3, num_words=3), sep='\n')

CPU times: user 24.1 s, sys: 261 ms, total: 24.4 s
Wall time: 13.6 s
(0, '0.011*"tax" + 0.008*"u" + 0.008*"company"')
(1, '0.010*"plant" + 0.008*"market" + 0.006*"amazon"')
(2, '0.006*"inc" + 0.006*"jan" + 0.005*"median"')


In [41]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

### Five-topic Model


In [42]:
%time ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50) #5 topics
print(*ldamodel.print_topics(num_topics=5, num_words=5), sep='\n')

CPU times: user 17.1 s, sys: 61.5 ms, total: 17.1 s
Wall time: 8.64 s
(0, '0.015*"jan" + 0.011*"sphere" + 0.010*"inc" + 0.010*"amazon" + 0.010*"seattle"')
(1, '0.024*"market" + 0.007*"industry" + 0.007*"report" + 0.006*"new" + 0.005*"also"')
(2, '0.005*"2" + 0.004*"caterpillar" + 0.003*"space" + 0.003*"last" + 0.003*"made"')
(3, '0.010*"tax" + 0.009*"company" + 0.008*"u" + 0.006*"city" + 0.005*"share"')
(4, '0.017*"plant" + 0.010*"case" + 0.005*"wardian" + 0.005*"terrarium" + 0.005*"operation"')


In [43]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

### Ten-topic Model

In [44]:
%time ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)
print(*ldamodel.print_topics(num_topics=10, num_words=5), sep='\n')

CPU times: user 20.3 s, sys: 317 ms, total: 20.7 s
Wall time: 12 s
(0, '0.021*"blade" + 0.018*"skid" + 0.017*"pusher" + 0.015*"snow" + 0.013*"rubber"')
(1, '0.015*"2018" + 0.012*"et" + 0.011*"jan" + 0.008*"am" + 0.007*"2017"')
(2, '0.012*"tax" + 0.010*"plant" + 0.009*"u" + 0.007*"company" + 0.006*"case"')
(3, '0.017*"caterpillar" + 0.015*"share" + 0.014*"company" + 0.010*"stock" + 0.007*"rating"')
(4, '0.015*"operation" + 0.014*"share" + 0.014*"cart" + 0.013*"jan" + 0.011*"topper"')
(5, '0.007*"year" + 0.007*"fit" + 0.007*"sold" + 0.007*"excavator" + 0.007*"bucket"')
(6, '0.016*"jan" + 0.014*"sphere" + 0.014*"inc" + 0.014*"amazon" + 0.013*"seattle"')
(7, '0.059*"market" + 0.016*"report" + 0.016*"industry" + 0.014*"analysis" + 0.013*"state"')
(8, '0.022*"market" + 0.009*"vehicle" + 0.008*"global" + 0.008*"turbocharger" + 0.007*"industry"')
(9, '0.013*"median" + 0.013*"city" + 0.012*"estimate" + 0.012*"university" + 0.010*"2017"')


In [45]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

## Conclusion:

After try different N, i think the best N for this topic is 5.
* First Group
(0, '0.015*"jan" + 0.011*"sphere" + 0.010*"inc" + 0.010*"amazon" + 0.010*"seattle"')
The first group seems like is talking about Amazon is opening something in their HQ setattle. This topic is soly about amazon.
* Second Group
(1, '0.024*"market" + 0.007*"industry" + 0.007*"report" + 0.006*"new" + 0.005*"also"')
The second group seems like is about the vehicle market group industry analysis report and forecast.
* Third Group
(2, '0.005*"2" + 0.004*"caterpillar" + 0.003*"space" + 0.003*"last" + 0.003*"made"')
The third group is pretty hard to tell, It seems like it is talking about the connection in between catepillar and the game(could be the dungeon game), or some other Fantasy topics.
* Fourth Group
(3, '0.010*"tax" + 0.009*"company" + 0.008*"u" + 0.006*"city" + 0.005*"share"')
The fourth group is talking about the tax related tasks about caterpillar in 2017 and potentially tax related problem with China.

* Fifth Group
(4, '0.017*"plant" + 0.010*"case" + 0.005*"wardian" + 0.005*"terrarium" + 0.005*"operation"')
The fifth group is about the victorian garden taking care of plants, it looks like a plant guide.